#**CONSULTING & ANALYTICS CLUB, IIT GUWAHATI**
##**CAPSTONE PROJECT**

## AUTHOR: ANUJ CHAUDHARY


**Generative Text for Customer Support Automation**

Project Overview: Develop an AI-powered system to automate customer support interactions using generative models like GPT-3.5.

**Use Cases:**

**Automated Response Generation:**

Problem Statement: Customer support teams are overwhelmed by repetitive inquiries that could be handled by automated systems.
Solution: Implement a generative AI model to automatically generate accurate and context-aware responses to common customer queries, reducing the load on human agents and improving response times.

**Personalized Customer Engagement:**

Problem Statement: Customers expect personalized interactions that cater to their specific needs and preferences.
Solution: Use generative AI to create personalized engagement messages based on customer data and interaction history, enhancing customer satisfaction and loyalty.


In [1]:
pip install transformers torch pandas


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv'

data = pd.read_csv(file_path)
print("Successfully Imported Data!")
data.head()

Mounted at /content/drive
Successfully Imported Data!


,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [4]:
import pandas as pd

df = pd.DataFrame(data)
# Save to CSV
file_path = "customer_support_data.csv"
df.to_csv("customer_support_data.csv", index=False)


In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

# Load your customer support data
data = pd.read_csv('customer_support_data.csv')
data.head()


,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [10]:
# Function to load dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset



In [11]:
# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
# Prepare the dataset and data collator
train_dataset = load_dataset("train.txt", tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [13]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.7 MB/s eta 0:00:00


In [14]:
!pip uninstall -y accelerate transformers
!pip install accelerate transformers[torch] --quiet

Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 32.0 MB/s eta 0:00:00


In [15]:
import accelerate
import transformers
import torch

print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)

Accelerate version: 0.32.1
Transformers version: 4.41.2
Torch version: 2.3.0+cu121


In [16]:
from packaging import version
import accelerate

required_version = "0.21.0"
installed_version = accelerate.__version__

if version.parse(installed_version) >= version.parse(required_version):
    print(f"Accelerate version {installed_version} is compatible.")
else:
    raise ImportError(
        f"Accelerate version {installed_version} is not compatible. "
        f"Please install accelerate>={required_version}."
    )

Accelerate version 0.32.1 is compatible.


In [17]:
!pip install transformers torch --quiet


In [18]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)



In [19]:
!pip install pyarrow==10.0.1 datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 10.0.1 which is incompatible.


In [20]:
!pip uninstall -y pyarrow datasets cudf-cu12

Found existing installation: pyarrow 10.0.1
Uninstalling pyarrow-10.0.1:
  Successfully uninstalled pyarrow-10.0.1
Found existing installation: datasets 2.11.0
Uninstalling datasets-2.11.0:
  Successfully uninstalled datasets-2.11.0
Found existing installation: cudf-cu12 24.4.1
Uninstalling cudf-cu12-24.4.1:
  Successfully uninstalled cudf-cu12-24.4.1


In [21]:
%pip install --upgrade pip



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [22]:

!pip install pyarrow==14.0.1 datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [23]:
# Initialize Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

# Assuming you are using a huggingface transformer model
from torch.optim import AdamW
learning_rate = 1e-5 # Set your desired learning rate here
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Prepare the model, optimizer, and dataloader for distributed training
model, optimizer, train_dataloader = accelerator.prepare(
    model,
    optimizer,
    train_dataset
)

In [25]:
num_train_epochs = 1
for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Check if 'batch' is a dictionary before proceeding
        if isinstance(batch, dict):
            print(f"Step {step} batch keys: {batch.keys()}")  # Print keys of the batch dictionary
            for key in batch:
                # Check if the item is a tensor before printing its shape
                if isinstance(batch[key], torch.Tensor):
                    print(f"{key} shape: {batch[key].shape}")
                else:
                    print(f"{key} is not a tensor")
        else:
            print(f"Step {step}: Batch is not a dictionary, it's a {type(batch)}")
        break

In [26]:
import torch

# Training loop
num_train_epochs = 1
for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Print batch shape
        print(f"Step {step} batch shape: {batch.shape}")

        # Ensure tensors have the correct dimensions
        if batch.dim() == 1:
            batch = batch.unsqueeze(0)

        # Move tensor to the appropriate device
        batch = batch.to(accelerator.device)

        # Assuming batch contains the input_ids and attention_mask
        # If batch is a tensor, you may need to generate attention masks
        attention_mask = torch.ones(batch.shape, device=batch.device)

        # Forward pass
        outputs = model(input_ids=batch, attention_mask=attention_mask, labels=batch) # Add labels for loss calculation
        # Check if model outputs a loss directly
        if hasattr(outputs, 'loss'):
            loss = outputs.loss
        else:
            # Calculate loss manually if needed (example with cross-entropy loss)
            loss_fn = torch.nn.CrossEntropyLoss()
            logits = outputs.logits  # Assuming your model outputs logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), batch.view(-1))
        # Backward pass and optimization
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

        if step % 100 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")



In [27]:
# Save the model
model.save_pretrained("./gpt2-customer-support")
tokenizer.save_pretrained("./gpt2-customer-support")

('./gpt2-customer-support/tokenizer_config.json',
 './gpt2-customer-support/special_tokens_map.json',
 './gpt2-customer-support/vocab.json',
 './gpt2-customer-support/merges.txt',
 './gpt2-customer-support/added_tokens.json')

In [28]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-customer-support")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-customer-support")

In [29]:
# Function to generate responses
def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [36]:
def generate_response(prompt):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate attention mask
    attention_mask = inputs['attention_mask']

    # Generate response
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=attention_mask,
        max_length=250,  # Increase the maximum length for a more detailed response
        min_length=50,   # Ensure a minimum length for the response
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevents repetition of 3-grams
        num_beams=5,  # Beam search for better output
        temperature=0.7,  # Control the randomness of predictions
        top_k=50,  # Consider the top 50 tokens by probability
        top_p=0.9  # Nucleus sampling - consider the top 90% of probability mass
    )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Example usage
customer_query = "What payment methods do you accept?"
response = generate_response(customer_query)
print("Response:", response)


Response: What payment methods do you accept?

We accept Visa, MasterCard, American Express, Discover, Discover Plus, and American Express.

How do I pay for my travel expenses?


You can pay for your travel expenses with a credit card, debit card, or money order. You can also pay with a check, money order, or any other form of payment.


Can I use my credit card or debit card to pay for travel expenses, such as food, lodging, or transportation?




Yes, you can use your credit card and debit card for all travel expenses. However, you cannot use your debit card or credit card for any travel expenses that are not directly related to your travel. For example, if you are traveling with a friend or family member, you may not be able to use your card to purchase food or lodging. You may also not be allowed to use the card to buy or rent a car. You must use your Visa or MasterCard to purchase and rent a vehicle. If you do not have a MasterCard or Visa card, you must use the credit or debit c